# Age Recognition Model Notebook using RESNET50

In [1]:
import tensorflow as tf
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.python.keras import models
from keras.applications.resnet import layers
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D

In [ ]:
# Consts

images = []
ages = []
# Please provide your own image path to the dataface with faces.
path = "C:/Users/mrsal/Github Repositories/Dataset/Age Recognition/UTKFace"

### Functions

In [2]:
# function to process images before training
def preprocessing(given_img):
    # given_img = cv2.cvtColor(given_img, cv2.COLOR_BGR2GRAY)  # converting image to grayscale.
    # given_img = cv2.equalizeHist(given_img)
    given_img = given_img / 255  # normalizing image.
    given_img = cv2.resize(given_img, (64, 64))  # resizing it.
    return given_img

In [ ]:
## function to create the model
def create_model(input_shape):
    base_model = tf.keras.applications.ResNet50(
        include_top=False, weights='imagenet', input_shape=input_shape
    )
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Conv2D(filters=128, kernel_size=3, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='linear')(x)
    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model

# Training the model

In [3]:
# Loading data
for image in os.listdir(path):
    img = cv2.imread(path + "/" + image)
    images.append(img)
    ages.append(float(image.split("_")[0]))
X_train, X_valid, y_train, y_valid = train_test_split(images, ages, test_size=0.33, random_state=42)
y_train = np.array(y_train)
y_valid = np.array(y_valid)

X_train = np.array([preprocessing(x) for x in X_train]).reshape(len(X_train), 64, 64, 3)
X_valid = np.array([preprocessing(x) for x in X_valid]).reshape(len(X_valid), 64, 64, 3)

In [6]:
# Creating the model and compile it
input_shape = (64, 64, 3)
model = create_model(input_shape)
model.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])
stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [7]:
# Fitting the model
## history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_valid, y_valid), callbacks=[stop])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_valid, y_valid))

Epoch 1/50
369/497 [=====================>........] - ETA: 23s - loss: 397.3486 - mean_squared_error: 397.3486

KeyboardInterrupt: 

In [ ]:
# Saving the model
model.save('RESNET50_MODEL_64.h5')

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_valid, y_valid)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

# Plot the training and validation loss/accuracy
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.show()


In [ ]:
# loading the model 
model = tf.keras.models.load_model("./RESNET50_MODEL_64.h5")